In [46]:
from langchain.llms import GooglePalm

In [47]:
api_key = "YOUR-API-KEY"

llm = GooglePalm(google_api_key = api_key, temperature=0.6)

In [4]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

Oh samosa, you are so delicious,
With your crispy shell and savory filling.
I love to eat you with my friends,
And I dream of you when I'm away.


In [6]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="codebasics_faqs.csv", source_column="prompt",encoding='latin')

data = loader.load()

In [41]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
#from langchain.vectorstores import Chroma

instructor_embeddings = HuggingFaceInstructEmbeddings()

vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)

pytorch_model.bin:  69%|######8   | 923M/1.34G [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [49]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents('How about job placement support?')
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [56]:
from langchain.prompts import PromptTemplate
# to avoid hallucinations
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [57]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
            llm = llm,
            chain_type= 'stuff',
            retriever=retriever,
            input_key='query',
            return_source_documents=True,
            chain_type_kwargs=chain_type_kwargs
            )

In [58]:
chain('Do you guys provide intership and also do you offer EMI payments?')

{'query': 'Do you guys provide intership and also do you offer EMI payments?',
 'result': 'Yes, We provide virtual internship and No',
 'source_documents': [Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample c

In [59]:
chain('Should I learn powerBI or tableau?')

{'query': 'Should I learn powerBI or tableau?',
 'result': 'This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner’s magic quadrant in BI for the last few years as the industry leader.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation 

In [60]:
chain('Do you have javascript course?')
# before prompt template, the answer was no

{'query': 'Do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
